TODO: implementera hdemucs i flax
- gör tester för sub-moduler
- testa ladda in modellen direkt via .pt-filen om möjligt
  - alternativt spara den i flax-format
- mät inferens-speed skillnad vs pytorch
  - naivt cpu
  - gpu
  - compiled

In [1]:
import torch
from torchaudio.models import HDemucs

state_dict_path = "models/hdemucs_high_trained.pt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sources = ["drums", "bass", "other", "vocals"]
model = HDemucs(sources=sources, nfft=4096, depth=6)

state_dict = torch.load(state_dict_path, weights_only=True)
model.load_state_dict(state_dict)
model.eval()

HDemucs(
  (freq_encoder): ModuleList(
    (0): _HEncLayer(
      (conv): Conv2d(4, 48, kernel_size=(8, 1), stride=(4, 1), padding=(2, 0))
      (norm1): Identity()
      (rewrite): Conv2d(48, 96, kernel_size=(1, 1), stride=(1, 1))
      (norm2): Identity()
      (dconv): _DConv(
        (layers): ModuleList(
          (0): Sequential(
            (0): Conv1d(48, 12, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): GroupNorm(1, 12, eps=1e-05, affine=True)
            (2): GELU(approximate='none')
            (3): Conv1d(12, 96, kernel_size=(1,), stride=(1,))
            (4): GroupNorm(1, 96, eps=1e-05, affine=True)
            (5): GLU(dim=1)
            (6): _LayerScale()
          )
          (1): Sequential(
            (0): Conv1d(48, 12, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
            (1): GroupNorm(1, 12, eps=1e-05, affine=True)
            (2): GELU(approximate='none')
            (3): Conv1d(12, 96, kernel_size=(1,), stride=(1,))
           

In [2]:
AUDIO = "./testaudio.wav"
import torchaudio

waveform, sample_rate = torchaudio.load(AUDIO, format="wav")

ref = waveform.mean(0)
waveform_n = (waveform - ref.mean()) / ref.std()

waveform_n = waveform_n.unsqueeze(0)
print(f"waveform_n.shape: {waveform_n.shape}") # (batch, channels, len)
result = model(waveform_n)
print(f"result.shape: {result.shape}") # (batch, sources, channels, length)


waveform_n.shape: torch.Size([1, 2, 184923])
result.shape: torch.Size([1, 4, 2, 184923])


In [3]:
import test_compare
from importlib import reload
reload(test_compare)

from test_compare import add_print_hook

add_print_hook(model)
res = model(waveform_n)


INFO:2025-03-25 18:28:50,567:jax._src.xla_bridge:945: Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
jax._src.xla_bridge - INFO - Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:2025-03-25 18:28:50,569:jax._src.xla_bridge:945: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache), 'libtpu.so' (no such file), '/usr/local/lib/libtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyld cache)
jax._src.xla_bridge - INFO - Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: dlopen(libtpu.so, 0x0001): tried: 'libtpu.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibtpu.so' (no such file), '/usr/lib/libtpu.so' (no such file, not in dyl

freq_emb.embedding, Conv1d input shape:	 torch.Size([1, 2, 184924]), norm: 608.1427612304688
freq_emb.embedding, Conv1d output shape:	 torch.Size([1, 48, 46231]), norm: 423.8492736816406
freq_emb.embedding, Identity input shape:	 torch.Size([1, 48, 46231]), norm: 423.8492736816406
freq_emb.embedding, Identity output shape:	 torch.Size([1, 48, 46231]), norm: 423.8492736816406
freq_emb.embedding, Conv1d input shape:	 torch.Size([1, 48, 46231]), norm: 271.998779296875
freq_emb.embedding, Conv1d output shape:	 torch.Size([1, 12, 46231]), norm: 908.998779296875
freq_emb.embedding, GroupNorm input shape:	 torch.Size([1, 12, 46231]), norm: 908.998779296875
freq_emb.embedding, GroupNorm output shape:	 torch.Size([1, 12, 46231]), norm: 500.3418884277344
freq_emb.embedding, GELU input shape:	 torch.Size([1, 12, 46231]), norm: 500.3418884277344
freq_emb.embedding, GELU output shape:	 torch.Size([1, 12, 46231]), norm: 269.49041748046875
freq_emb.embedding, Conv1d input shape:	 torch.Size([1, 12, 4

In [ ]:
import time
import numpy as np
from tqdm import tqdm

def time_model_forward():
    # Run forward pass multiple times and measure execution time
    n_runs = 10
    times = []
    
    for _ in tqdm(range(n_runs)):
        start = time.perf_counter()
        with torch.no_grad():
            _ = model(waveform_n) 
        end = time.perf_counter()
        times.append(end - start)
    
    times = np.array(times)
    avg_time = np.mean(times)
    std_time = np.std(times)
    
    print(f"Average forward pass time: {avg_time:.4f}s ± {std_time:.4f}s")
    print(f"Min time: {np.min(times):.4f}s")
    print(f"Max time: {np.max(times):.4f}s")

# Warm up run
with torch.no_grad():
    _ = model(waveform_n)

time_model_forward()


In [ ]:
from pprint import pprint
pprint(model.freq_emb)
pprint(model.freq_emb.scale)

model.freq_emb.weight.shape

import torch

x = torch.ones((512,), dtype=torch.int32)
print(x.shape)

y = model.freq_emb(x)
print(y.shape)

y[0]

In [ ]:
from importlib import reload
import demucs
reload(demucs)
from demucs import ScaledEmbedding

from flax import nnx
import jax.numpy as jnp

scaled_embedding = ScaledEmbedding(3, 3, rngs=nnx.Rngs(0))

print(scaled_embedding(jnp.array([0, 1, 2])))

print(scaled_embedding.embedding.embedding)

scaled_embedding.embedding.embedding = nnx.Param(torch.ones((3, 3)).detach().numpy())

print(scaled_embedding.embedding.embedding)

print(scaled_embedding(jnp.array([0, 1, 2])))


In [ ]:
nnx.state(scaled_embedding)

In [ ]:
import utils
reload(utils)
from utils import torch_module_to_params

params = torch_module_to_params(model.freq_emb)

print(params)


In [ ]:
model.state_dict()

In [ ]:
model_params = torch_module_to_params(model)
import jax

jax.tree_util.tree_map(lambda x: x.shape, model_params)

import flax
d = flax.traverse_util.unflatten_dict(model_params, sep=".")

d_jax = jax.tree_util.tree_map(lambda x: x.shape, d)
pprint(d_jax)

In [ ]:
from transformers.modeling_flax_pytorch_utils import convert_pytorch_sharded_state_dict_to_flax

convert_pytorch_sharded_state_dict_to_flax(model_params, model.params)

model_params["freq_emb.embedding.embedding"]



In [5]:
import flax.linen as nn
import jax.numpy as jnp
from flax import nnx

class Foo(nnx.Module):
  def __call__(self, x):
    return x

def my_interceptor1(next_fun, args, kwargs, context):
  print('calling my_interceptor1')
  return next_fun(*args, **kwargs)

foo = Foo()
with nn.intercept_methods(my_interceptor1):
  _ = foo(jnp.ones([1]))

AttributeError: module 'flax.nnx' has no attribute 'intercept_methods'